# Example Scenario 2: Validation and statistics on a subset of Wikidata

*Bob wants to extract a subset of Wikidata with the `member of (P463)` property, clean it, remove irrelevant columns (for this task), and compute its statistics, including centrality metrics.*

**Note on the expected running time:** Running this notebook takes under an hour on a Macbook Pro laptop with MacOS Catalina 10.15, a 2.3 GHz 8-Core Intel Core i9 processor, 2TB SSD disk, and 64 GB 2667 MHz DDR4 memory.

## Preparation (same as in Example 3)

To run this notebook, Bob would need the Wikidata edges file. We will work with version `20200405` of Wikidata. Presumably, this file is not present on Bob's laptop, so we need to download and unpack it first:
* please download the file [here](https://drive.google.com/file/d/1WIQIYXJC1IdSlPchtqz0NDr2zEEOz-Hb/view?usp=sharing)
* unpack it by running : `gunzip wikidata_edges_20200504.tsv.gz`

You are all set!

*Note*: Here we assume that the Wikidata file has already been transformed to KGTK format from Wikidata's `json.bz2` dump. This can be done with the following KGTK command (for demonstration purposes, we will skip this command, as its execution takes around 11 hours): `kgtk import-wikidata -i wikidata-20200504-all.json.bz2 --node wikidata_nodes_20200504.tsv --edge wikidata_edges_20200504.tsv -qual wikidata_qualifiers_20200504.tsv`

## Implementation in KGTK

We filter the data for all `P463` relations.

In [1]:
%%bash
kgtk filter -p ' ; P463 ; ' -i wikidata_edges_20200504.tsv > p463.tsv

Next, we clean it and remove columns that are not relevant for this use case:

In [2]:
%env ignore_cols=id,rank,node2;magnitude,node2;unit,node2;item,node2;lower,node2;upper,node2;entity-type,node2;longitude,node2;latitude,node2;date,node2;calendar,node2;precision

env: ignore_cols=id,rank,node2;magnitude,node2;unit,node2;item,node2;lower,node2;upper,node2;entity-type,node2;longitude,node2;latitude,node2;date,node2;calendar,node2;precision


In [3]:
%%bash
kgtk clean-data --error-limit 0 -i p463.tsv  / remove_columns -c "$ignore_cols" | grep . > graph.tsv

Finally, we compute graph statistics:

In [4]:
%%bash
kgtk graph-statistics --directed --degrees --pagerank --log p463_summary.txt -i graph.tsv > p463_stats.tsv

You can now inspect the individual node metrics in `p463_stats` or read the summary in `p463_summary.txt`.

For example, we learn that the mean degree is 2.45 and that the node with a highest PageRank is ORCID, Inc. (`Q19861084`).